In [19]:
import pandas as pd
import os

In [18]:
data_path = '/root/dialouge/data/'
train = pd.read_csv(os.path.join(data_path,'train.csv'))

NameError: name 'os' is not defined

## 요약문과 대화 길이 

* max_length 조절을 위해

In [10]:
# dialog 와 summary 각각의 모델 max_length 설정을 위한 길이 확인
import pandas as pd
import json
df = pd.read_csv('/root/dialogue/data/total_train.csv')


train_dialog_length = df['dialogue'].apply(lambda x: len(str(x)))
train_summary_length = df['summary'].apply(lambda x: len(str(x)))

print("대화 길이에 대한 정보")
print(train_dialog_length.describe())
print("=================")
print("요약문 길이에 대한 정보")
print(train_summary_length.describe())

대화 길이에 대한 정보
count    28826.000000
mean       351.743565
std        307.942919
min          3.000000
25%        173.000000
50%        316.000000
75%        468.000000
max      33043.000000
Name: dialogue, dtype: float64
요약문 길이에 대한 정보
count    28826.000000
mean        73.608236
std         37.897022
min          3.000000
25%         46.000000
50%         67.000000
75%         94.000000
max        478.000000
Name: summary, dtype: float64


## 자주 사용하는 단어 확인

* TF-IDF(Term Frequency - Inverse Document Frequency)

In [8]:
# 단어 토큰화, 명사 추출 
from konlpy.tag import Okt
k=Okt()

print('단어 토큰화 결과 ==>', k.morphs(train['dialogue'].iloc[0]))
# print('명사 추출 결과 ==>', k.nouns(train['dialogue'].iloc[0]))

단어 토큰화 결과 ==> ['#Person1', '#:', '안녕하세요', ',', '스미스', '씨', '.', '저', '는', '호킨스', '의사', '입니다', '.', '오늘', '왜', '오셨나요', '?', '\n', '#Person2', '#:', '건강검진', '을', '받는', '것', '이', '좋을', '것', '같아서요', '.', '\n', '#Person1', '#:', '그렇군요', ',', '당신', '은', '5년', '동안', '건강검진', '을', '받지', '않았습니다', '.', '매년', '받아야', '합니다', '.', '\n', '#Person2', '#:', '알', '고', '있습니다', '.', '하지만', '아무', '문제', '가', '없다면', '왜', '의사', '를', '만나러', '가야', '하나요', '?', '\n', '#Person1', '#:', '심각한', '질병', '을', '피하', '는', '가장', '좋은', '방법', '은', '이를', '조기', '에', '발견', '하는', '것', '입니다', '.', '그러니', '당신', '의', '건강', '을', '위해', '최소한', '매년', '한', '번은', '오세요', '.', '\n', '#Person2', '#:', '알겠습니다', '.', '\n', '#Person1', '#:', '여기', '보세요', '.', '당신', '의', '눈', '과', '귀', '는', '괜찮아', '보입니다', '.', '깊게', '숨', '을', '들이', '쉬세요', '.', '스미스', '씨', ',', '담배', '피우시나요', '?', '\n', '#Person2', '#:', '네', '.', '\n', '#Person1', '#:', '당신', '도', '알다시피', ',', '담배', '는', '폐암', '과', '심장병', '의', '주요', '원인', '입니다', '.', '정말로', '끊으셔야', '합니다', '.', '

## 개인정보 마스킹 확인

In [39]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# 1. 마스킹 정보 확인
'''
전화번호 → #PhoneNumber#
주소 → #Address#
생년월일 → #DateOfBirth#
여권번호 → #PassportNumber#
사회보장번호 → #SSN#
신용카드 번호 → #CardNumber#
차량 번호 → #CarNumber#
이메일 주소 → #Email#
'''
# 정규표현식 사용하기 
import re


df = pd.read_csv('/root/dialouge/output1.csv')
def reg_masking(text):
  pattern = r"#\w+#"  # ## 사이의 값을 추출하는 정규식 패턴
  masked = re.findall(pattern, text)
  return masked

train_set = df['dialogue'].apply(lambda x:str(set(reg_masking(x))))
train_set.head(100)

In [ ]:
# 2.발화자 확인
import re
def reg_person(text):
  pattern = r"#\w+\d#"  # ## 사이의 값을 추출하는 정규식 패턴 > special token 으로 tokenizer에 추가
  masked = re.findall(pattern, text)
  return masked

train_person = train['dialogue'].apply(lambda x:set(reg_person(x)))
train_person


In [29]:
import pandas as pd
import re

# CSV 파일 읽기
df = pd.read_csv('/root/dialouge/output2.csv')

# 특정 컬럼에서 #과 # 사이의 값을 추출하는 함수 정의
def extract_between_hashes(text):
    if pd.isna(text):  # NaN 값 체크
        return None
    matches = re.findall(r'#(.*?)#', text)
    return matches[0] if matches else None  # 첫 번째 매치된 값 반환, 없으면 None

# 'summary' 컬럼에 대해 #과 # 사이의 값을 추출
if 'summary' in df.columns:  # 'summary' 컬럼이 있는지 확인
    df['summary'] = df['summary'].apply(extract_between_hashes)
    # 결과 출력
    print(df['summary'].unique())
else:
    print("'summary' 컬럼이 존재하지 않습니다.")


[None 'Person2' 'Person1']


In [46]:
#4. special token 추가
# train_11716 -> ##person#으로 되어 있음

from transformers import AutoTokenizer

unique_token = [
    "#PhoneNumber#", "#Address#", "#DateOfBirth#", 
    "#PassportNumber#", "#SSN#", "#CardNumber#", 
    "#CarNumber#", "#Email#","#Person1#", "#Person2#", "#Person3#"
]

tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")
special_tokens_dict = {'additional_special_tokens': unique_token}
tokenizer.add_special_tokens(special_tokens_dict)

tokenizer.additional_special_tokens, tokenizer.additional_special_tokens_ids

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


(['#Email#',
  '#DateOfBirth#',
  '#CarNumber#',
  '#PassportNumber#',
  '#Person2#',
  '#Address#',
  '#Person1#',
  '#CardNumber#',
  '#Person3#',
  '#PhoneNumber#',
  '#SSN#'],
 [30000, 30001, 30002, 30003, 30004, 30005, 30006, 30007, 30008, 30009, 30010])

## AEDA

In [27]:
!pip install mecab-python3

  Obtaining dependency information for mecab-python3 from https://files.pythonhosted.org/packages/9b/7e/29dd338e62dfbfaa0b656442d2a31a6d822c048a247dabae5a22608ae88a/mecab_python3-1.0.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 1.8 MB/s eta 0:00:00a 0:00:01m


In [ ]:
import MeCab
mecab = MeCab.Tagger()


In [16]:
a =train['summary'][:4]
a[1]

'파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후 리키에게 백신을 접종했다.'

In [24]:
from koeda import EasyDataAugmentation

def augment_text_data_with_EDA(text,repetition):
    """입력된 문장에 대해서 EDA를 통해 데이터 증강"""
    eda = EasyDataAugmentation(
        morpheme_analyzer="Okt"
        )

    result = eda(text,p=(0.5, 0.5, 0.5, 0.5), repetition=repetition)

    # 증강 결과 출력
    print("원문: " , text)
    print("--"*100)
    for i in range(repetition):
        print(f"증강문{i+1}: ", result[i])
    # return result

In [32]:
from textattack.augmentation import EasyDataAugmenter
import pandas as pd

# AEDA 증강기를 초기화합니다.
eda_augmenter = EasyDataAugmenter()

# 대화문을 증강하는 함수입니다.
def augment_dialogue(dialogue: str, augmenter) -> str:
    return augmenter.augment(dialogue)

# 원본 대화문 데이터
data = [
    {
        "fname": "train_0",
        "dialogue": "#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요? #Person2#: 건강검진을 받는 것이 좋을 것 같아서요. #Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다. #Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요? #Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요. #Person2#: 알겠습니다. #Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요? #Person2#: 네. #Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. #Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다. #Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다. #Person2#: 알겠습니다, 감사합니다, 의사선생님.",
        "summary": "스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.",
        "topic": "건강검진 받기"
    },
    # 다른 데이터 샘플...
]

# 대화문 증강 예제
for entry in data:
    original_dialogue = entry["dialogue"]
    augmented_dialogue = augment_dialogue(original_dialogue, eda_augmenter)
    
    print("Original Dialogue:\n", original_dialogue)
    print("Augmented Dialogue:\n", augmented_dialogue)
    print("-" * 50)


textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /data/ephemeral/home/nltk_data...


Original Dialogue:
 #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요? #Person2#: 건강검진을 받는 것이 좋을 것 같아서요. #Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다. #Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요? #Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요. #Person2#: 알겠습니다. #Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요? #Person2#: 네. #Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. #Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다. #Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다. #Person2#: 알겠습니다, 감사합니다, 의사선생님.
Augmented Dialogue:
 ['#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요? #Person2#: 건강검진을 받는 것이 좋을 것 같아서요. #Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다. #Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요? #Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요. #Person2#: 알겠습니다. #Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요? #Person2#: 네. #Person1

In [1]:
!pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/5e/4d/affea11bd85ca69d9fdd15567495bb9088ac1c37498c95cb42d9ecd984ed/openai-1.43.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/56/95/9377bcb415797e44274b51d46e3249eba641711cf3348050f76ee7b15ffc/httpx-0.27.2-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/41/6a/c038077509d67fe876c724bfe9ad15334593851a7def0d84518172bdd44a/jiter-0.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pydantic<3,>=1.9.0 from https://files.pythonhosted.org/packages/1f/fa/b7f815b8c9ad021c07f88875b601222e

In [49]:
from openai import OpenAI  # Ensure this version is installed: openai==1.2.0
import pandas as pd

# Initialize the OpenAI client
client = OpenAI(
    api_key="api",  # Replace with your actual API key
    base_url="https://api.upstage.ai/v1/solar"
)

# Function to translate text using the solar-1-mini-translate-enko model
def translate_text(text,summary, translate_summary):
    try:
        response = client.chat.completions.create(
            model="solar-1-mini-translate-enko",
            messages=[
                {
                    "role": "user",
                    "content": translate_summary
                },
                {
                    "role": "assistant",
                    "content": summary
                },
                {
                    "role": "user",
                    "content": text
                }
            ],
            stream=False  # Set to True if you want to use streaming
        )
        # Return the translated text
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error translating text: {e}")
        return None
    


df = pd.read_csv("/root/dialouge/code/prediction/output.csv") 
df_train = pd.read_csv('/root/dialouge/data/translate/train_translated.csv')

# Translate each summary
df["translated_summary"] = df["translated_summary","summary","text"].apply(translate_text)

df.to_csv("output_.csv", index=False)
print("saved to output_.csv.")


Translation completed and saved to output_.csv.


In [6]:
from openai import OpenAI  # Ensure this version is installed: openai==1.2.0
import pandas as pd

# Initialize the OpenAI client
client = OpenAI(
    api_key="up_CNqmlQHjivsljcVtjTtfMJlfoT7jE",  # Replace with your actual API key
    base_url="https://api.upstage.ai/v1/solar"
)

# Function to translate text using the solar-1-mini-translate-enko model
def translate_text(text):
    try:
        response = client.chat.completions.create(
            model="solar-1-mini-translate-enko",
            messages=[
                {
                    "role": "user",
                    "content": "#Person2#'s mom lost her job. #Person2# doesn't want her mom to be depressed. #Person1# suggests #Person2# look for job leads on the internet."
                },
                {
                    "role": "assistant",
                    "content": "#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 않기를 바란다. #Person1#은 #Person2#에게 인터넷에서 일자리 정보를 찾아보는 것을 제안한다."
                },
                {
                    "role": "user",
                    "content": "#Person1# asks #Person2# for advice on how to pack a bag for a visit to this uncle's house next Saturday."
                },
                {
                    "role": "assistant",
                    "content": "#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 #Person2#에게 조언을 구합니다."
                },
                {
                    "role": "user",
                    "content": "데이브는 #Person2#가 그가 웹트래커에서 일하고 있다는 것을 추론할 수 있다는 사실에 놀란다."
                },
                {
                    "role": "assistant",
                    "content": "Dave is surprised that #Person2# can deduce that he is working for WebTracker."
                },
                {
                    "role": "user",
                    "content": "Bill tells #Person1# that he found out his roommate is Brain Locker."
                },
                {
                    "role": "assistant",
                    "content": "빌은 #Person1#에게 자신의 룸메이트가 브래인 로커라는 것을 알게 되었다고 말한다."
                },
                {
                    "role": "user",
                    "content": text
                }
            ],
            stream=False  # Set to True if you want to use streaming
        )
        # Return the translated text
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error translating text: {e}")
        return None

df = pd.read_csv("/root/dialouge/code/prediction/output.csv") 

# Translate each summary
df["translated_summary"] = df["summary"].apply(translate_text)

df.to_csv("output_.csv", index=False)
print("saved to output_.csv.")


saved to output_.csv.


In [7]:
import pandas as pd

df = pd.read_csv('/root/dialouge/output_.csv')

df = df.drop(columns='summary', axis=0)

df = df.rename(columns={"translated_summary":"summary"})

df.to_csv('output2.csv', index=False)

In [2]:
import pandas as pd
df = pd.read_csv('/root/dialogue/baseline/prediction/output.csv')
df.summary = df.summary.str.strip()
df.to_csv('output3.csv', index=False)

In [11]:
import json
import pandas as pd
with open('/root/dialouge/data/train.json', 'r', encoding='utf-8') as file:
                data = json.load(file)
                df = pd.DataFrame(data)

df

,id,summary,dialogue
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella...."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...
...,...,...,...
15545,13829423,Carla's date for graduation is on June 4th. Di...,Carla: I've got it...\r\nDiego: what?\r\nCarla...
15546,13727710,Bev is going on the school trip with her son. ...,"Gita: Hello, this is Beti's Mum Gita, I wanted..."
15547,13829261,Greg cheated on Julia. He apologises to her. R...,"Julia: Greg just texted me\r\nRobert: ugh, del..."
15548,13680226,Marry broke her nail and has a party tomorrow....,"Marry: I broke my nail ;(\r\nTina: oh, no!\r\n..."


In [12]:
df = df.rename(columns = {'summary':'translated_summary', 'dialogue':'translated_dialogue'})

df

,id,translated_summary,translated_dialogue
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella...."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...
...,...,...,...
15545,13829423,Carla's date for graduation is on June 4th. Di...,Carla: I've got it...\r\nDiego: what?\r\nCarla...
15546,13727710,Bev is going on the school trip with her son. ...,"Gita: Hello, this is Beti's Mum Gita, I wanted..."
15547,13829261,Greg cheated on Julia. He apologises to her. R...,"Julia: Greg just texted me\r\nRobert: ugh, del..."
15548,13680226,Marry broke her nail and has a party tomorrow....,"Marry: I broke my nail ;(\r\nTina: oh, no!\r\n..."


In [ ]:
df1 = pd.read_csv('/root/dialouge/data/translate/train_translated.csv')
df1

In [13]:
last_fname = df1['fname'].iloc[-1]  # 'train_0'
last_index = int(last_fname.split('_')[1])
next_index = last_index + 1

# Create new entries for df1
df_new = pd.DataFrame({
    'fname': [f'train_{i}' for i in range(next_index, next_index + len(df))],
    'dialogue': df['translated_dialogue'].tolist(),
    'summary': df['translated_summary'].tolist(),
    'topic': [''] * len(df),  # Adjust as needed
    'translated_dialogue': df['translated_dialogue'].tolist(),
    'translated_summary': df['translated_summary'].tolist()
})

# Append df_new to df1
df1_updated = pd.concat([df1, df_new], ignore_index=True)